# Introduction

<center><img src="https://i.imgur.com/9hLRsjZ.jpg" height=400></center>

This dataset was scraped from [nextspaceflight.com](https://nextspaceflight.com/launches/past/?page=1) and includes all the space missions since the beginning of Space Race between the USA and the Soviet Union in 1957!

### Install Package with Country Codes

In [150]:
#%pip install iso3166

### Upgrade Plotly

Run the cell below if you are working with Google Colab.

In [151]:
#%pip install --upgrade plotly

### Import Statements

In [152]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# These might be helpful:
from iso3166 import countries
from datetime import datetime, timedelta

### Notebook Presentation

In [153]:
pd.options.display.float_format = '{:,.2f}'.format

### Load the Data

In [154]:
df_data = pd.read_csv('mission_launches.csv')

# Preliminary Data Exploration

* What is the shape of `df_data`? 
* How many rows and columns does it have?
* What are the column names?
* Are there any NaN values or duplicates?

In [155]:
df_data.shape

(4324, 9)

In [156]:
df_data.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Organisation', 'Location', 'Date',
       'Detail', 'Rocket_Status', 'Price', 'Mission_Status'],
      dtype='object')

In [157]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4324 entries, 0 to 4323
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0.1    4324 non-null   int64 
 1   Unnamed: 0      4324 non-null   int64 
 2   Organisation    4324 non-null   object
 3   Location        4324 non-null   object
 4   Date            4324 non-null   object
 5   Detail          4324 non-null   object
 6   Rocket_Status   4324 non-null   object
 7   Price           964 non-null    object
 8   Mission_Status  4324 non-null   object
dtypes: int64(2), object(7)
memory usage: 304.2+ KB


Testing for NaN and duplicated values:

In [158]:
df_data.isna().values.any()

True

In [159]:
df_data.duplicated().values.any()

False

## Data Cleaning - Check for Missing Values and Duplicates

Consider removing columns containing junk data. 

In [160]:
df_data[df_data.isna().any(axis=1)]

,Unnamed: 0.1,Unnamed: 0,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
7,7,7,CASC,"LC-101, Wenchang Satellite Launch Center, China","Thu Jul 23, 2020 04:41 UTC",Long March 5 | Tianwen-1,StatusActive,NaN,Success
13,13,13,IAI,"Pad 1, Palmachim Airbase, Israel","Mon Jul 06, 2020 01:00 UTC",Shavit-2 | Ofek-16,StatusActive,NaN,Success
28,28,28,VKS RF,"Site 43/4, Plesetsk Cosmodrome, Russia","Fri May 22, 2020 07:31 UTC",Soyuz 2.1b/Fregat-M | Cosmos 2546,StatusActive,NaN,Success
31,31,31,ExPace,"Site 95, Jiuquan Satellite Launch Center, China","Tue May 12, 2020 01:16 UTC",Kuaizhou 1A | Xingyun-2 01 (Wuhan) & 02,StatusActive,NaN,Success
...,...,...,...,...,...,...,...,...,...
4319,4319,4319,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA","Wed Feb 05, 1958 07:33 UTC",Vanguard | Vanguard TV3BU,StatusRetired,NaN,Failure
4320,4320,4320,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA","Sat Feb 01, 1958 03:48 UTC",Juno I | Explorer 1,StatusRetired,NaN,Success
4321,4321,4321,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA","Fri Dec 06, 1957 16:44 UTC",Vanguard | Vanguard TV3,StatusRetired,NaN,Failure
4322,4322,4322,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan","Sun Nov 03, 1957 02:30 UTC",Sputnik 8K71PS | Sputnik-2,StatusRetired,NaN,Success


In [161]:
df_data.Price = df_data.Price.str.replace(",", "")
clean_df = df_data.dropna()
clean_df.shape

(964, 9)

## Descriptive Statistics

In [162]:
df_data = df_data.drop(["Unnamed: 0.1", "Unnamed: 0"], axis=1)

In [163]:
df_data.Price = np.where(df_data.Price == df_data.Price.isnull().values.any(), df_data.Price, df_data.Price.astype(float))

In [164]:
clean_df.Price = pd.to_numeric(clean_df.Price)

In [165]:
df_data['Price'].describe()

count    964.00
unique    56.00
top      450.00
freq     136.00
Name: Price, dtype: float64

# Number of Launches per Company

Create a chart that shows the number of space mission launches by organisation.

In [166]:
launch_per_organisations = df_data['Organisation'].value_counts()
launch_per_organisations = pd.DataFrame(launch_per_organisations.values, launch_per_organisations.index, columns=['Total Launches'])
launch_per_organisations

,Total Launches
RVSN USSR,1777
Arianespace,279
CASC,251
General Dynamics,251
NASA,203
VKS RF,201
US Air Force,161
ULA,140
Boeing,136
Martin Marietta,114


In [167]:
fig = px.bar(
    launch_per_organisations,
    x='Total Launches',
    y=launch_per_organisations.index,
    height=800,
    color='Total Launches',
    labels={'index': 'Company', 'y': 'Number of Launches'},
    color_continuous_scale=px.colors.sequential.Viridis,
    orientation="h"
)

fig.update_layout(
    title="Number of launches per Company",
    xaxis_title="Company",
    yaxis_title="Total",
    yaxis={'categoryorder':'total ascending', "dtick": 1}
)

fig.show()

# Number of Active versus Retired Rockets

How many rockets are active compared to those that are decomissioned? 

In [168]:
df_rocket = df_data['Rocket_Status'].value_counts()
df_rocket = pd.DataFrame(df_rocket.values, df_rocket.index, columns=['Count'])
df_rocket

,Count
StatusRetired,3534
StatusActive,790


In [169]:
rocket_fig = px.bar(
    df_rocket,
    x="Count",
    y=df_rocket.index,
    color="Count",
    orientation="h",
    color_continuous_scale=px.colors.sequential.RdBu_r,
    height=250
)

rocket_fig.update_layout(
    title="Active rockets Vs. Decomissioned rockets",
    yaxis=dict(title="Status", categoryorder="total ascending"),
    xaxis_title="Count"
)

rocket_fig.update(layout_coloraxis_showscale=False)

rocket_fig.show()

# Distribution of Mission Status

How many missions were successful?
How many missions failed?

In [170]:
df_missions_status = df_data['Mission_Status'].value_counts()
df_missions_status = pd.DataFrame(df_missions_status.values, df_missions_status.index, columns=['Count'])
df_missions_status

,Count
Success,3879
Failure,339
Partial Failure,102
Prelaunch Failure,4


In [171]:
missions_status_fig = px.pie(
    df_missions_status,
    values="Count",
    names=df_missions_status.index,
    hole=0.5,
    color_discrete_sequence=px.colors.sequential.Agsunset
)

missions_status_fig.update_layout(
    title="Distribution of Mission Status"
)

missions_status_fig.show()

# How Expensive are the Launches? 

Create a histogram and visualise the distribution. The price column is given in USD millions (careful of missing values). 

In [172]:
fig_price = px.histogram(
    clean_df,
    "Price"
)

fig_price.update_layout(
    title="Distribution of launches prices",
    yaxis_title=None,
    xaxis_title="Price (in USD millions)"
)

fig_price.show()


# Use a Choropleth Map to Show the Number of Launches by Country

* Create a choropleth map using [the plotly documentation](https://plotly.com/python/choropleth-maps/)
* Experiment with [plotly's available colours](https://plotly.com/python/builtin-colorscales/). I quite like the sequential colour `matter` on this map. 
* You'll need to extract a `country` feature as well as change the country names that no longer exist.

Wrangle the Country Names

You'll need to use a 3 letter country code for each country. You might have to change some country names.

* Russia is the Russian Federation
* New Mexico should be USA
* Yellow Sea refers to China
* Shahrud Missile Test Site should be Iran
* Pacific Missile Range Facility should be USA
* Barents Sea should be Russian Federation
* Gran Canaria should be USA


You can use the iso3166 package to convert the country names to Alpha3 format.

In [173]:
df_data['Country'] = df_data['Location'].str.rsplit(",", 1).str[-1].str.strip()

In [174]:
df_data.loc[(df_data.Country == "Russia"), "Country"] = "Russian Federation"
df_data.loc[(df_data.Country == "New Mexico"), "Country"] = "USA"
df_data.loc[(df_data.Country == "Yellow Sea"), "Country"] = "China"
df_data.loc[(df_data.Country == "Shahrud Missile Test Site"), "Country"] = "Iran"
df_data.loc[(df_data.Country == "Pacific Missile Range Facility"), "Country"] = "USA"
df_data.loc[(df_data.Country == "Barents Sea"), "Country"] = "Russian Federation"
df_data.loc[(df_data.Country == "Gran Canaria"), "Country"] = "USA"

df_data.Country.unique()

array(['USA', 'China', 'Kazakhstan', 'Japan', 'Israel', 'New Zealand',
       'Russian Federation', 'Iran', 'France', 'India', 'North Korea',
       'Pacific Ocean', 'South Korea', 'Brazil', 'Kenya', 'Australia'],
      dtype=object)

In [175]:
df_data.loc[(df_data.Country == "Iran", "Country")] = "IRN"
df_data.loc[(df_data.Country == "North Korea", "Country")] = "PRK"

In [176]:
def get_country_name(x):
    if x != "IRN" and x != "PRK":
        try:
            return countries.get(x).alpha3
        except KeyError:
            return 'Unknown'
    else:
        return x

In [177]:
df_data.Country = df_data.Country.apply(lambda x: get_country_name(x))

In [178]:
launch_per_country_df = df_data.groupby("Country").count()
launch_per_country_df

,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status
Country,,,,,,,
AUS,6,6,6,6,6,0,6
BRA,3,3,3,3,3,0,3
CHN,269,269,269,269,269,159,269
FRA,303,303,303,303,303,95,303
IND,76,76,76,76,76,67,76
IRN,14,14,14,14,14,0,14
ISR,11,11,11,11,11,0,11
JPN,126,126,126,126,126,40,126
KAZ,701,701,701,701,701,46,701


In [179]:
launch_per_country_df = df_data.groupby("Country").count()
launch_per_country_df = launch_per_country_df.drop(launch_per_country_df[launch_per_country_df.columns.difference(["Organisation"])], axis=1)
launch_per_country_df = launch_per_country_df.rename(columns={"Organisation" : "Total Launches"})
launch_per_country_df

,Total Launches
Country,
AUS,6
BRA,3
CHN,269
FRA,303
IND,76
IRN,14
ISR,11
JPN,126
KAZ,701


In [180]:
fig_map = px.choropleth(
    launch_per_country_df,
    locations=launch_per_country_df.index,
    color="Total Launches",
    color_continuous_scale=px.colors.sequential.Tealgrn[3:],
    range_color=(0,1500),
    width=900,
    height=500
)

fig_map.update_geos(
    visible=False,
    showcountries=True,
    countrycolor="#455A64",
    bgcolor="#323130"
)

fig_map.update_layout(
    paper_bgcolor="#323130",
    title="Map world of launches per country",
    titlefont=dict(color="white"),
    coloraxis_colorbar=dict(
        title="Number of Launches", 
        titlefont=dict(color="white"), 
        tickfont=dict(color="white")
    )
)

fig_map.show()

# Use a Choropleth Map to Show the Number of Failures by Country


In [181]:
df_data['Mission_Status_Binary'] = np.where(df_data.Mission_Status != "Success", "Failure", "Success")
df_failures = df_data.groupby(["Country", "Mission_Status_Binary"]).count()
df_failures = df_failures.query('Mission_Status_Binary == "Failure"')
df_failures = df_failures.drop(df_failures[df_failures.columns.difference(["Organisation"])], axis=1)
df_failures = df_failures.rename(columns={"Organisation": "Failures"})
df_failures = df_failures.reset_index()
df_failures

,Country,Mission_Status_Binary,Failures
0,AUS,Failure,3
1,BRA,Failure,3
2,CHN,Failure,25
3,FRA,Failure,18
4,IND,Failure,13
5,IRN,Failure,9
6,ISR,Failure,2
7,JPN,Failure,13
8,KAZ,Failure,93
9,NZL,Failure,2


In [182]:
fig_failure = px.choropleth(
    df_failures,
    locations="Country",
    color="Failures",
    color_continuous_scale=px.colors.sequential.Reds[3:],
    range_color=(0, 160),
    width=900,
    height=500
)

fig_failure.update_geos(
    visible=False,
    showcountries=True,
    countrycolor="#455A64",
    bgcolor="#323130"
)

fig_failure.update_layout(
    paper_bgcolor="#323130",
    title="Map world of total failures launches per country",
    titlefont=dict(color="white"),
    coloraxis_colorbar=dict(
        title="Total failures", 
        titlefont=dict(color="white"), 
        tickfont=dict(color="white")
    )
)

fig_failure.show()

# Create a Plotly Sunburst Chart of the countries, organisations, and mission status. 

In [183]:
fig = px.sunburst(
    df_data,
    path=['Country', 'Organisation', 'Mission_Status'],
    color="Country",
    width=1000,
    height=600
)

fig.update_layout(
    title="Sunburst Chart of countries, organisations and status missions",
    titlefont=dict(color="white"),
    paper_bgcolor="#323130"
)

fig.show()

# Analyse the Total Amount of Money Spent by Organisation on Space Missions

In [184]:
df_amount = clean_df.groupby("Organisation").agg({'Price': 'sum'})
df_amount = df_amount.reset_index()
df_amount

,Organisation,Price
0,Arianespace,"16,345.00"
1,Boeing,"1,241.00"
2,CASC,"6,340.26"
3,EER,20.00
4,ESA,37.00
5,Eurockot,543.40
6,ExPace,28.30
7,ILS,"1,320.00"
8,ISRO,"2,177.00"
9,JAXA,168.00


In [185]:
fig_amount = px.bar(
    df_amount,
    x='Price',
    y="Organisation",
    color='Price',
    orientation="h",
    labels={'x': 'Organisation', 'y': 'Total price'},
    color_continuous_scale=px.colors.sequential.algae[2:],
    height=600
)

fig_amount.update_layout(
    title="Total amount per Organisation",
    xaxis=dict(title="Amount (in USD millions)"),
    yaxis_title="Organisation",
    yaxis=dict(categoryorder='total ascending', dtick=1)
)

fig_amount.show()

**NASA** organisation seems to be by far the organization that spends the most

# Analyse the Amount of Money Spent by Organisation per Launch

In [186]:
df_mean_amount = clean_df.groupby("Organisation").agg({'Price': 'mean'})
df_mean_amount = df_mean_amount.rename(columns={'Price': 'Average Price'})
df_mean_amount


,Average Price
Organisation,
Arianespace,170.26
Boeing,177.29
CASC,40.13
EER,20.00
ESA,37.00
Eurockot,41.80
ExPace,28.30
ILS,101.54
ISRO,32.49


In [187]:
fig_mean_amount = px.bar(
    df_mean_amount,
    x="Average Price",
    y=df_mean_amount.index,
    color='Average Price',
    orientation="h",
    labels={'index': 'Organisation', 'y': 'Average Price'},
    color_continuous_scale=px.colors.sequential.Blues[5:],
    text_auto=".2s",
    height=600,
    range_color=(0, 5000)
)

fig_mean_amount.update_layout(
    title="Average amount spent by Organisation per Launch",
    xaxis_title="Average Amount (in USD millions)",
    yaxis_title="Organisation",
    yaxis={'categoryorder':'total ascending', 'dtick': 1}
)

fig_mean_amount.update_traces(
    textfont_size=9, 
)

fig_mean_amount.show()

However, the most average amount spent per Launch is by **RVSN USSR** organisation by far

# Chart the Number of Launches per Year

In [188]:
df_data['Year Launch'] = df_data.Date.str.split(",").str[1].str.strip().str.split(" ").str[0]
df_data['Year Launch'] = pd.DatetimeIndex(df_data['Year Launch']).year
df_launch_per_year = df_data.groupby("Year Launch").count()
df_launch_per_year = df_launch_per_year.drop(df_launch_per_year[df_launch_per_year.columns.difference(["Organisation"])], axis=1).rename(columns={'Organisation': 'Count'})
df_launch_per_year

,Count
Year Launch,
1957,3
1958,28
1959,20
1960,39
1961,52
...,...
2016,90
2017,92
2018,117


In [189]:
fig_launch_per_year = px.bar(
    df_launch_per_year,
    x=df_launch_per_year.index,
    y="Count",
    color='Count',
    color_continuous_scale=px.colors.sequential.algae,
)

fig_launch_per_year.update_layout(
    title="Number of Launches per Year",
    xaxis_title="Year",
    yaxis_title="Number of Launches",
)

fig_launch_per_year.show()

# Chart the Number of Launches Month-on-Month until the Present

Which month has seen the highest number of launches in all time? Superimpose a rolling average on the month on month time series chart. 

In [190]:
def get_formatted_date(x):
    if 'UTC' in x:
        x = x.rsplit(" ", 2)[0]
        return pd.to_datetime(x, format="%a %b %d, %Y")
    else:
        pd.to_datetime(x, format="%a %b %d, %Y")

In [244]:
df_data["Date Format"] = df_data['Date']
df_data['Date Format'] = df_data['Date Format'].apply(lambda x: get_formatted_date(x))
df_data.head()

,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status,Country,Mission_Status_Binary,Year Launch,Date Format
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.00,Success,USA,Success,2020,2020-08-07
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success,CHN,Success,2020,2020-08-06
2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success,USA,Success,2020,2020-08-04
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.00,Success,KAZ,Success,2020,2020-07-30
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.00,Success,USA,Success,2020,2020-07-30


In [245]:
# All launches dates:
df_launch_per_date = df_data.groupby("Date Format").count()
df_launch_per_date = df_launch_per_date.drop(df_launch_per_date[df_launch_per_date.columns.difference(["Organisation"])], axis=1).rename(columns={"Organisation": "Count"})
df_launch_per_date

,Count
Date Format,
1957-10-04,1
1957-11-03,1
1957-12-06,1
1958-02-01,1
1958-02-05,1
...,...
2020-07-25,1
2020-07-30,2
2020-08-04,1


In [246]:
# Launches per months:
df_month_on_month = df_launch_per_date
df_month_on_month.index = df_month_on_month.index.to_period("M")
df_month_on_month = df_month_on_month.groupby(df_month_on_month.index).count()
df_month_on_month = df_month_on_month.resample("M").sum()
df_month_on_month = df_month_on_month.fillna(0)
df_month_on_month

,Count
Date Format,
1957-10,1.00
1957-11,1.00
1957-12,1.00
1958-01,0.00
1958-02,2.00
...,...
2020-04,3.00
2020-05,9.00
2020-06,6.00


In [247]:
df_month_on_month['SMA1Y'] = df_month_on_month.rolling(12).mean()
df_month_on_month

,Count,SMA1Y
Date Format,,
1957-10,1.00,NaN
1957-11,1.00,NaN
1957-12,1.00,NaN
1958-01,0.00,NaN
1958-02,2.00,NaN
...,...,...
2020-04,3.00,7.25
2020-05,9.00,7.33
2020-06,6.00,7.33


In [248]:
fig_month_on_month = px.line(
    df_month_on_month,
    x=df_month_on_month.index.to_timestamp(),
    y="Count",
    title="Number of Launches Month-on-Month until the Present"
)

fig_month_on_month.update_traces(line_color='#456987')

fig_month_on_month.add_scatter(
    x=df_month_on_month.index.to_timestamp(),
    y=df_month_on_month["SMA1Y"],
    name="1 year Moving Average",
    line=dict(color="aliceblue")
)

fig_month_on_month.update_layout(
    paper_bgcolor="#212121",
    plot_bgcolor="#212121",
    legend=dict(font=dict(color="white")),
    xaxis=dict(color="white", title="Time"),
    yaxis=dict(color="white", title="Launches"),
    titlefont=dict(color="white")
)

fig_month_on_month.update_xaxes(gridcolor='#3C3C3C')
fig_month_on_month.update_yaxes(gridcolor="#3C3C3C")

fig_month_on_month.show()

The end of 60's to 80's seems to be the period with the largest number of launches, specially in **April 1968** which is the month with the highest number of launches of all time

# Launches per Month: Which months are most popular and least popular for launches?

Some months have better weather than others. Which time of year seems to be best for space missions?

In [249]:
df_month_on_month = df_month_on_month.reset_index()
df_month_on_month

,Date Format,Count,SMA1Y
0,1957-10,1.00,NaN
1,1957-11,1.00,NaN
2,1957-12,1.00,NaN
3,1958-01,0.00,NaN
4,1958-02,2.00,NaN
...,...,...,...
750,2020-04,3.00,7.25
751,2020-05,9.00,7.33
752,2020-06,6.00,7.33
753,2020-07,11.00,7.42


In [271]:
df_monthly = pd.DataFrame().assign(Months=df_month_on_month['Date Format'], Count=df_month_on_month['Count'])
df_monthly['Months'] = df_monthly['Months'].dt.strftime('%Y-%B')
df_monthly['Months'] = df_monthly['Months'].str.split('-').str[-1]
df_monthly = df_monthly.groupby("Months").mean()
df_monthly = df_monthly.sort_values("Count", ascending=False)
df_monthly

,Count
Months,
December,6.06
June,5.52
October,5.48
September,5.24
April,5.22
August,5.13
July,4.95
March,4.95
February,4.84


In [319]:
fig_monthly = px.bar(
    df_monthly,
    x="Count",
    y=df_monthly.index,
    color="Count",
    color_continuous_scale=px.colors.sequential.Viridis[1:],
    title="Launches per Month"
)

fig_monthly.update_layout(
    yaxis=dict(categoryorder="total ascending", color="white"),
    xaxis=dict(color="white"),
    titlefont=dict(color="white"),
    paper_bgcolor="#212121",
    plot_bgcolor="#212121"
)

fig_monthly.update(layout_coloraxis_showscale=False)
fig_monthly.update_xaxes(showgrid=False, zeroline=False)
fig_monthly.update_yaxes(showgrid=False, zeroline=False)

fig_monthly.show()

# How has the Launch Price varied Over Time? 

Create a line chart that shows the average price of rocket launches over time. 

# Chart the Number of Launches over Time by the Top 10 Organisations. 

How has the dominance of launches changed over time between the different players? 

# Cold War Space Race: USA vs USSR

The cold war lasted from the start of the dataset up until 1991. 

## Create a Plotly Pie Chart comparing the total number of launches of the USSR and the USA

Hint: Remember to include former Soviet Republics like Kazakhstan when analysing the total number of launches. 

## Create a Chart that Shows the Total Number of Launches Year-On-Year by the Two Superpowers

## Chart the Total Number of Mission Failures Year on Year.

## Chart the Percentage of Failures over Time

Did failures go up or down over time? Did the countries get better at minimising risk and improving their chances of success over time? 

# For Every Year Show which Country was in the Lead in terms of Total Number of Launches up to and including including 2020)

Do the results change if we only look at the number of successful launches? 

# Create a Year-on-Year Chart Showing the Organisation Doing the Most Number of Launches

Which organisation was dominant in the 1970s and 1980s? Which organisation was dominant in 2018, 2019 and 2020? 